Data loading and preproccessing
First of, we will laod the data and take the select few related to data science and computers, with one outlier being Memory article.

In [2]:
import json

target_articles = {}
# Loading articles from dev
with open('./data/stanford-question-answering-dataset/dev-v1.1.json', 'r', encoding='utf-8') as f:
    squad_data_dev = json.load(f)
for i, article in enumerate(squad_data_dev['data']):
    if i==4 or i==19 :
        target_articles[article['title']] = article['paragraphs']
# Loading articles from train
with open('./data/stanford-question-answering-dataset/train-v1.1.json', 'r', encoding='utf-8') as f2:
    squad_data_train = json.load(f2)
for i, article in enumerate(squad_data_train['data']):
    if i in {30, 62, 104, 124, 149, 157, 177, 229, 293, 295, 330, 333, 389, 393} :
        target_articles[article['title']] = article['paragraphs']

In [3]:
for article_title, paragraphs in target_articles.items():
    print(f"""Article: {article_title}""")

Article: Computational_complexity_theory
Article: Packet_switching
Article: Wayback_Machine
Article: Web_browser
Article: Computer
Article: Computer_security
Article: ASCII
Article: Macintosh
Article: Memory
Article: Data_compression
Article: USB
Article: Unicode
Article: IBM
Article: Software_testing
Article: Database
Article: Printed_circuit_board


After loading in all the articles, we can prepare text for chunking and questions for testing the retrieval

In [4]:
from langchain_core.documents import Document

docs_for_splitter = []
questions_ground_truth = []
p_idx=0
for article_title, paragraphs in target_articles.items():
    for _, para_data in enumerate(paragraphs):
        context_text = para_data['context']
        
        # Creating documents with needed metadata
        doc = Document(
            page_content=context_text,
            metadata={
                "para_id": p_idx,
                "article": article_title
            }
        )
        docs_for_splitter.append(doc)
        
        # Extracting questions and linking them to doc that should be retrieved for them
        for qa in para_data['qas']:
            questions_ground_truth.append({
                "question": qa['question'],
                "ground_truth_para_id": p_idx,
                # "qa_id": qa['id'],
                # "article": article_title
            })
        p_idx+=1

############################################################### <br>
BELOW IS WORK IN PROGRESS

Splitting paragraphs into docs that will be embeded. Here I used the same tokenizer from the mixedbread mode with RecursiveCharacterTextSplitter, this way I hope to utilize the whole context window and minimize number of sentences that are split across chunks.

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from langchain_text_splitters import RecursiveCharacterTextSplitter

tokenizer = AutoTokenizer.from_pretrained("mixedbread-ai/mxbai-embed-large-v1")
splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer,chunk_size=512, chunk_overlap=75)
splits = splitter.split_documents(docs_for_splitter)


Embedding the documents

In [6]:
from langchain_community.vectorstores import Chroma
embed = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")
vectorstore = Chroma.from_documents(documents=splits,embedding=embed)
retriever = vectorstore.as_retriever()

README.md: 0.00B [00:00, ?B/s]

In [20]:
id = 1003
print(f"Question: {questions_ground_truth[id]['question']}")
print(f"Target paragraph id: {questions_ground_truth[id]['ground_truth_para_id']}")

print(retriever.invoke(questions_ground_truth[id]['question']))

Question: What did Regis McKenna call the "1984" ad that was aired during the Super Bowl?
Target paragraph id: 270
[Document(metadata={'article': 'Macintosh', 'para_id': 270}, page_content='After the Lisa\'s announcement, John Dvorak discussed rumors of a mysterious "MacIntosh" project at Apple in February 1983. The company announced the Macintosh 128K—manufactured at an Apple factory in Fremont, California—in October 1983, followed by an 18-page brochure included with various magazines in December. The Macintosh was introduced by a US$1.5 million Ridley Scott television commercial, "1984". It most notably aired during the third quarter of Super Bowl XVIII on January 22, 1984, and is now considered a "watershed event" and a "masterpiece." Regis McKenna called the ad "more successful than the Mac itself." "1984" used an unnamed heroine to represent the coming of the Macintosh (indicated by a Picasso-style picture of the computer on her white tank top) as a means of saving humanity from 

In [21]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 7})

In [ ]:
from tqdm import tqdm # For a nice progress bar

def evaluate_retrieval(questions, retriever, k_values=[1, 5, 7]):
    hits = {k: 0 for k in k_values}
    total_questions = len(questions)

    print(f"Starting evaluation on {total_questions} questions...")

    for q_data in tqdm(questions):
        question = q_data['question']
        target_id = q_data['ground_truth_para_id']
        
        retrieved_docs = retriever.invoke(question) 
        
        retrieved_ids = [doc.metadata.get('para_id') for doc in retrieved_docs]

        for k in k_values:
            # Looking only at the top k retrieved IDs
            if target_id in retrieved_ids[:k]:
                hits[k] += 1

    print("\n--- Retrieval Performance ---")
    for k in k_values:
        accuracy = (hits[k] / total_questions) * 100
        print(f"Hit Rate@{k}: {accuracy:.2f}% ({hits[k]}/{total_questions})")

evaluate_retrieval(questions_ground_truth, retriever)

Starting evaluation on 2265 questions...


100%|██████████| 2265/2265 [08:24<00:00,  4.49it/s]


--- Retrieval Performance ---
Hit Rate@1: 73.38% (1662/2265)
Hit Rate@5: 91.96% (2083/2265)
Hit Rate@7: 93.77% (2124/2265)


In [ ]:
def evaluate_retrieval2(questions, retriever, k_values=[1, 5, 7]):
    hits = {k: 0 for k in k_values}
    total_questions = len(questions)

    print(f"Starting evaluation on {total_questions} questions...")

    for q_data in tqdm(questions):
        question = q_data['question']
        target_id = q_data['ground_truth_para_id']
        
        retrieved_docs = retriever.invoke(f"""Represent this sentence for searching relevant passages: {question}""") 
        
        retrieved_ids = [doc.metadata.get('para_id') for doc in retrieved_docs]

        for k in k_values:
            # Looking only at the top k retrieved IDs
            if target_id in retrieved_ids[:k]:
                hits[k] += 1

    print("\n--- Retrieval Performance ---")
    for k in k_values:
        accuracy = (hits[k] / total_questions) * 100
        print(f"Hit Rate@{k}: {accuracy:.2f}% ({hits[k]}/{total_questions})")

evaluate_retrieval2(questions_ground_truth, retriever)

Starting evaluation on 2265 questions...


100%|██████████| 2265/2265 [08:59<00:00,  4.20it/s]


--- Retrieval Performance ---
Hit Rate@1: 71.48% (1619/2265)
Hit Rate@5: 90.99% (2061/2265)
Hit Rate@7: 93.07% (2108/2265)
